In [0]:
!unzip -q Data\ Science.zip

In [0]:
!pip3 install catboost

     |████████████████████████████████| 62.9MB 102kB/s 


In [0]:
import pandas as pd
import datetime

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randInt
from scipy.stats import uniform as sp_randFloat
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [0]:
data = pd.read_csv('TRAIN_PREPARED.csv')

Y = data.label.values
X = data.drop('label', axis = 1)

X['first_date'] = X['first_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp())
X['last_date'] = X['last_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp())
X['cut_date'] = X['cut_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp()) 
X = X.drop('user', axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 32)
X_train, X_eval, Y_train, Y_eval = train_test_split(X_train, Y_train, test_size = 0.12, random_state = 32)

print("Train size: {} Test size: {}".format(len(X_train), len(X_test)))

Train size: 142757 Test size: 18025


In [0]:
model = CatBoostClassifier(iterations=2000, task_type="GPU", devices='0:1', eval_metric = 'Recall' )

model = CatBoostClassifier(iterations=2000, task_type="GPU", devices='0:1', eval_metric = 'Recall', depth = 14, learning_rate = 0.1, l2_leaf_reg = 4, random_strength = 0.5)
model.fit(X_train, Y_train, eval_set = (X_test, Y_test), verbose = 500)




0:	learn: 0.1739530	test: 0.1660937	best: 0.1660937 (0)	total: 519ms	remaining: 17m 17s
500:	learn: 0.9998649	test: 0.9379029	best: 0.9379029 (498)	total: 3m 11s	remaining: 9m 34s
1000:	learn: 0.9999189	test: 0.9454233	best: 0.9458530 (970)	total: 6m 20s	remaining: 6m 19s
1500:	learn: 0.9999189	test: 0.9467125	best: 0.9471422 (1125)	total: 9m 26s	remaining: 3m 8s
1999:	learn: 0.9999189	test: 0.9473571	best: 0.9477869 (1985)	total: 12m 31s	remaining: 0us
bestTest = 0.94778685
bestIteration = 1985
Shrink model to first 1986 iterations.


In [0]:
test = pd.read_csv('TEST_PREPARED.csv')

test['first_date'] = test['first_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp())
test['last_date'] = test['last_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp())
test['cut_date'] = test['cut_date'].apply(lambda t: datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').timestamp()) 

users = test.pop('user')
preds = model.predict(test)
submit = {'user': users, 'prediction': preds}
submit = pd.DataFrame.from_dict(submit)
submit.to_csv('submit.csv', index = False)
